#### 검증을 위해 테스트세트만 사용했을 때 문제점
- 테스트 세트를 사용해 자꾸 성능을 확인하면 모델을 수정하면 결국 모델이 점점 테스트 세트에 맞추어져 범용적으로 사용이 어려움    
     
#### 해결책 : 검증세트를 사용
- 검증세트 : 테스트세트는 마지막에 1번만 사용하기 위해 훈련세트에서 다시 떼어 낸 데이터 세트   

#### 검증세트를 만드느라 줄어든 훈련세트 문제를 해결할 수 있는 방법 : 교차검증
- 교차검증 : 훈련세트를 여러 폴드로 나눈 다음 한 폴드가 검증 세트의 역할을 하고 나머지 폴드에서는 모델을 훈련, 이런 식으로 모든 폴드에 대한 검증 점수를 얻어 평균하여 검증점수를 구하는 방법

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import uniform, randint       # scipy : 적분, 보간, 선형 대수, 확률 등을 포함한 수치 계산 전용 라이브러리

In [ ]:
# 검증셋 떼어내기
wine = pd.read_csv('https://bit.ly/wine_csv_data',)
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42) # 테스트데이터를 떼어낸 데이터에서 검증세트를 떼어내기

print('훈련세트개수:',sub_input.shape,'\n테스트세트개수:', test_input.shape,'\n검증세트개수:', val_input.shape) # 테스트세트랑 검증세트의 데이터 개수가 다르다
print()

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print('훈련셋점수:',dt.score(sub_input, sub_target))
print('검증셋점수:',dt.score(val_input, val_target))   # 과대적합

훈련세트개수: (4157, 3) 
테스트세트개수: (1300, 3) 
검증세트개수: (1040, 3)

훈련셋점수: 0.9971133028626413
검증셋점수: 0.864423076923077


In [ ]:
# k-fold cross validation : 훈련세트를 k개의 부분으로 나눠서 교차 검증을 수행
scores = cross_validate(dt, train_input, train_target)  # cv매개변수에서 폴드 수를 바꿀 수 있다, cross_validate는 훈련세트를 섞지 않음 - 섞으려면 분할기를 이용 (회귀 : KFold, 분류 : StratifiedKFold)
print(scores)                                           # fit_time : 모델을 훈련하는 시간, score_time : 모델을 검증하는 시간, test_score : 검증 점수
print()                                          
print('검증점수 :',np.mean(scores['test_score']))
scores_2 = cross_validate(dt, train_input, train_target, cv = StratifiedKFold()) # cv에서 분할기와 폴드 개수 지정 가능
print('StratifiedKFold분할기를 이용한 검증점수:',np.mean(scores_2['test_score']))
print()

splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores_10 = cross_validate(dt, train_input, train_target, cv=splitter)
print('10-폴드 교차검증 시에 검증점수:',np.mean(scores_10['test_score']))

{'fit_time': array([0.01009846, 0.01052356, 0.01238537, 0.01034522, 0.01005769]), 'score_time': array([0.0012188 , 0.00118208, 0.00127602, 0.00114536, 0.00122023]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}

검증점수 : 0.855300214703487
StratifiedKFold분할기를 이용한 검증점수: 0.855300214703487

10-폴드 교차검증 시에 검증점수: 0.8574181117533719


#### 하이퍼파라미터 튜닝 
- 최적의 모델을 만들기 위하여 하이퍼파라미터를 바꿔서 모델을 학습하는 과정
- 그리드서치 : 사이킷런에서 제공하는 도구로 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행가능
- 랜덤서치 : 하이퍼파라미터의 값의 범위나 간격을 미리 정하기 어려운 경우 확률분포 객체를 전달하여 하이퍼파라미터 탐색과 교차 검증을 한 번에 수행

In [ ]:
params = {'min_impurity_decrease': [0.0001,0.0002,0.0003,0.0004,0.0005]}           # DecisionTreeClassifier의 min_impurity_decrease : 불순도 감소 최소량
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs=-1)    # n_jobs : CPU 코어 수를 지정 (-1일 때 모든 코어를 사용)
gs.fit(train_input, train_target)
bt = gs.best_estimator_                                                            # 최적의 파라미터를 찾아 훈련한 모델이 best_estimator_에 저장되어 있음
print('bestmodel_trainscore:',dt.score(train_input, train_target))
print('bestmodel_testscore:',dt.score(test_input, test_target))
print('bestparameter:', gs.best_params_)                                           # 최적의 파라미터가 best_params_에 저장되어 있음
print('각 매개변수에서 수행한 교차검증의 평균점수', params)
print(gs.cv_results_['mean_test_score'])
print()
# best_index = np.argmax(gs.cv_results_['mean_test_score'])    # np.argmax : 가장 큰 인덱스 값 추출
# print(gs.cv_results_['params'][best_index])                  

params_3 = {'min_impurity_decrease' : np.arange(0.0001,0.001,0.0001),
            'max_depth': range(5,20,1),
            'min_samples_split': range(2,100,10)
            }                                                             # min_samples_split : 노드를 나누기 위한 최소 샘플 수
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params_3, n_jobs=-1)
gs.fit(train_input, train_target)
print('best parameter\n',gs.best_params_)
print('best score:',np.max(gs.cv_results_['mean_test_score']))

bestmodel_trainscore: 0.970559938426015
bestmodel_testscore: 0.8569230769230769
bestparameter: {'min_impurity_decrease': 0.0001}
각 매개변수에서 수행한 교차검증의 평균점수 {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]

best parameter
 {'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
best score: 0.8683865773302731


In [ ]:
# 랜덤서치 : 값의 범위나 간격을 미리 정하기 어려운 경우 확률분포 객체를 전달
# 주어진 범위에서 고르게 값을 뽑는 클래스 : uniform(실수값), randint(정수값) 
params_r = {'min_impurity_decrease' : uniform(0.0001,0.001),
            'max_depth': randint(20,50),
            'min_samples_split': randint(2,25),
            'min_samples_leaf': randint(1,25),
            }      
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params_r, n_iter=100, n_jobs=-1,random_state=42)  # n_iter: 반복횟수
rs.fit(train_input, train_target)
print('best parameter',rs.best_params_)
print('best validation score:',np.max(rs.cv_results_['mean_test_score']))
dt_r = rs.best_estimator_
print('test score:',dt_r.score(test_input, test_target))

best parameter {'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}
best validation score: 0.8695428296438884
test score: 0.86
